# Statistics Canada exports data

Source: [Statistics Canada](https://open.canada.ca/data/en/dataset/b1126a07-fd85-4d56-8395-143aba1747a4)

## 1. Import, tidy and transform data

In [1]:
import pandas as pd

material = 'plastics' # 'plastics', 'paper'
csv_file = 'Trade-Exports-Chp-39.csv' if material == 'plastics' \
    else 'Trade-Exports-Chp-47.csv'
quantity = 'kg' if material == 'plastics' else 'metric_tonnes'
codes = [391510, 391520, 391530, 391590] if material == 'plastics' \
    else [470710, 470720, 470730, 470790]

In [2]:
df = pd.read_csv(
        f'data/raw/canada-statistics-canada/Trade-Exports-Yr2017-2018/{csv_file}',
        encoding='latin-1',
        usecols=[0, 2, 3, 4, 6, 7, 8]
    ) \
    .rename({
        'HS_CODE/CODE_SH': 'hs_code',
        'COUNTRY/PAYS': 'country_code',
        'STATE/ÉTAT': 'state_code',
        'GEO/GEO': 'geo',
        'QUANTITY/QUANTITÉ': 'quantity',
        'YEAR/ANNÉE': 'year',
        'MONTH/MOIS': 'month'
    }, axis=1) \
    .pipe(lambda df: df.query(
        'hs_code in @codes & \
        country_code != 999 & \
        state_code == 1000 & \
        geo == 1'
    )) \
    .assign(period = lambda x:
        pd.to_datetime(x['year'].map(str) + '-' + x['month'].map(str) + '-01'),
        kg = lambda x: x['quantity'] * 1000 if material == 'paper' else x['quantity']
    ) \
    .groupby(['country_code', 'period'], as_index=False)['kg'].sum() \
    .sort_values(['country_code', 'period']) \
    .reset_index(drop=True)

print(len(df))

df.head()

340


,country_code,period,kg
0,9,2017-01-01,8511164
1,9,2017-02-01,8029145
2,9,2017-03-01,10681859
3,9,2017-04-01,9727159
4,9,2017-05-01,10250167


## 2. Join to country names and ISO codes

Country/area codes source: Statistics Canada

In [3]:
df_countries = pd.read_excel(
        'data/raw/canada-statistics-canada/C616N006_CtryDesc_201807.xlsx',
        header=5,
        usecols=[0, 2, 5],
        keep_default_na=False # Necessary because the ISO 2 code for Namibia is ‘NA’
    ) \
    .rename({
        'ISO Country Code': 'iso2',
        'ITD Country Code': 'country_code'
    }, axis=1) \
    .drop_duplicates(['iso2', 'country_code']) \
    .reset_index(drop=True)

print(len(df_countries))

df_countries.head()

261


,iso2,country_code,Country English Description
0,AD,156,Andorra
1,AE,328,United Arab Emirates
2,AF,542,Afghanistan
3,AG,832,Antigua and Barbuda
4,AI,830,Anguilla


In [4]:
df_joined = df.copy() \
    .merge(
        df_countries,
        'left',
        'country_code'
    ) \
    .drop('country_code', axis=1) \
    .rename({'Country English Description': 'country_name'}, axis=1) \
    .reset_index(drop=True)

print(len(df_joined))

df_joined.head()

340


,period,kg,iso2,country_name
0,2017-01-01,8511164,US,United States
1,2017-02-01,8029145,US,United States
2,2017-03-01,10681859,US,United States
3,2017-04-01,9727159,US,United States
4,2017-05-01,10250167,US,United States


## 3. Check data types and nulls

In [5]:
df_joined.dtypes

period          datetime64[ns]
kg                       int64
iso2                    object
country_name            object
dtype: object

In [6]:
df_joined.isnull().any()

period          False
kg              False
iso2            False
country_name    False
dtype: bool

## 4. Spot check, e.g. December 2017 exports to US

In [7]:
test = df_joined.query(
    'period.dt.year == 2017 & \
    period.dt.month == 12 & \
    country_name == "United States"'
)

test['kg'].sum()

8067622

## 5. Identify countries importing zero waste

In [8]:
null_countries = df_joined.copy() \
    .groupby('iso2', as_index=False).sum() \
    .query('kg == 0') \
    ['iso2'].tolist()

print(len(null_countries))

null_countries[0:10]

0


[]

## 6. Write monthly data to CSV

In [9]:
df_monthly = df_joined.copy() \
    .query('iso2 not in @null_countries') \
    .assign(exporter = 'CA')
    
df_monthly.head()

,period,kg,iso2,country_name,exporter
0,2017-01-01,8511164,US,United States,CA
1,2017-02-01,8029145,US,United States,CA
2,2017-03-01,10681859,US,United States,CA
3,2017-04-01,9727159,US,United States,CA
4,2017-05-01,10250167,US,United States,CA


In [10]:
df_monthly[['period', 'iso2', 'country_name', 'exporter', 'kg']].to_csv(
    f'data/processed/canada-statistics-canada/ca-{material}-exports-monthly.csv',
    index=False
)

## 7. Compare H1 2017 to H1 2018

In [11]:
df_h1 = df_joined.copy() \
    .query(
        'iso2 not in @null_countries & \
        period.dt.year in [2017, 2018] & \
        period.dt.month in [1, 2, 3, 4, 5, 6]'
    ) \
    .pivot_table(
        values='kg',
        index='country_name',
        columns=df_joined['period'].dt.year,
        aggfunc='sum'
    ) \
    .assign(pct_change = lambda x: (x[2018] - x[2017]) / x[2017]) \
    .assign(h1_2017_proportion = lambda x: x[2017] / x[2017].sum()) \
    .assign(h1_2018_proportion = lambda x: x[2018] / x[2018].sum()) \
    .assign(pct_change_proportional = lambda x:
        x['h1_2018_proportion'] - x['h1_2017_proportion']) \
    .sort_values('pct_change_proportional')

df_h1.head()

period,2017,2018,pct_change,h1_2017_proportion,h1_2018_proportion,pct_change_proportional
country_name,,,,,,
China,20398059.0,874396.0,-0.957133,0.196520,0.010279,-0.186241
Hong Kong,17859425.0,2566798.0,-0.856278,0.172062,0.030175,-0.141888
Spain,327317.0,158692.0,-0.515173,0.003153,0.001866,-0.001288
Japan,65044.0,18176.0,-0.720558,0.000627,0.000214,-0.000413
Singapore,41774.0,2070.0,-0.950448,0.000402,0.000024,-0.000378


## 8. Write H1 comparison data to CSV

In [12]:
# df_h1.to_csv(
#     f'data/processed/canada-statistics-canada/ca-{material}-exports-h1.csv',
#     index=False
# )